In [1]:
import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed
from collections import defaultdict
import requests


In [ ]:
FILTER_DIRECT = True
FILTER_PRICE = 350
URL = "https://partners.api.skyscanner.net/apiservices/v3/flights/indicative/search"
HEADERS = {'x-api-key': 'sh428739766321522266746152871799'}


def get_place_from_id(id: str, places: dict) -> str:
    place = places[id]
    if 'AIRPORT' in place['type']:
        return place['name']
    return f"{place['name']}_{place['type']}"


def check_origin_flights_from_date(year: int, month: int, day: int, origin_airport: str) -> tuple[list, list]:
    body = {
        'query': {
            'currency': 'PLN',
            'locale': 'pl-PL',
            'market': 'PL',
            # 'dateTimeGroupingType': 'DATE_TIME_GROUPING_TYPE_BY_DATE',
            'queryLegs': [
                {
                    'originPlace': {
                        'queryPlace': {
                            'iata': origin_airport,
                        },
                    },
                    'destinationPlace': {
                        'anywhere': True,
                    },
                    "fixedDate": {
                        "year": year,
                        "month": month,
                        "day": day

                    },
                },
            ],
        },
    }

    response = requests.post(URL, headers=HEADERS, json=body)
    response = response.json()['content']['results']

    if FILTER_DIRECT:
        response['quotes'] = {k: v for k, v in response['quotes'].items() if v['isDirect']}

    if FILTER_PRICE:
        response['quotes'] = {k: v for k, v in response['quotes'].items() if int(v['minPrice']['amount']) <= FILTER_PRICE}
    flights = []
    for flight_info in response['quotes'].values():
        assert flight_info['minPrice']['unit'] == 'PRICE_UNIT_WHOLE'
        flights.append(
            dict(
                price=int(flight_info['minPrice']['amount']),
                origin_place=get_place_from_id(flight_info['outboundLeg']['originPlaceId'], response['places']),
                destination_place=get_place_from_id(
                    flight_info['outboundLeg']['destinationPlaceId'], response['places']
                ),
                when=flight_info['outboundLeg']['departureDateTime'],
                destination_place_id=int(flight_info['outboundLeg']['destinationPlaceId']),
            )
        )
    return flights


def all_weekday_dates_in_year(year: int, weekday: int = 4, month: int = 1, day: int = 1) -> list[datetime.date]:
    # January 1st of the given year
    date = datetime.date(year, month, day)

    # Find the first Saturday of the year
    while date.weekday() != weekday:  # Saturday is represented by 5
        date += datetime.timedelta(days=1)

    # Loop through the Saturdays and print them
    dates = []
    while date.year == year:
        dates.append(date)
        date += datetime.timedelta(weeks=1)
    return dates


current_date = datetime.date.today()

def get_origin_flights(current_date: datetime.date):
    all_fridays = all_weekday_dates_in_year(2023, weekday=4, month=current_date.month, day=current_date.day)
    all_saturdays = all_weekday_dates_in_year(2023, weekday=5, month=current_date.month, day=current_date.day)
    all_dates = [*all_fridays, *all_saturdays]
    
    all_flights = []
    with ThreadPoolExecutor() as executor:
        futures = [
            executor.submit(check_origin_flights_from_date, f_date.year, f_date.month, f_date.day, origin)
            for f_date in all_dates for origin in ['WRO', 'POZ']
        ]
        for future in as_completed(futures):
            all_flights.extend(future.result())
    return all_flights

def group_flights_by_destination(flights: list[dict]) -> dict:
    grouped_flights = defaultdict(list)
    for flight in flights:
        grouped_flights[flight['destination_place']].append(flight)
    return grouped_flights

all_origin_flights = get_origin_flights(current_date)
grouped_origin_flights = group_flights_by_destination(all_origin_flights)

In [ ]:
def check_destination_to_origin_flights_from_date(year: int, month: int, day: int, destination_airport_id: int, origin_iata: str) -> tuple[list, list]:
    body = {
        'query': {
            'currency': 'PLN',
            'locale': 'pl-PL',
            'market': 'PL',
            'queryLegs': [
                {
                    'originPlace': {
                        'queryPlace': {
                            'entityId': destination_airport_id,
                        },
                    },
                    'destinationPlace': {
                        'queryPlace': {
                            'iata': origin_iata,
                        },
                    },
                    "fixedDate": {
                        "year": year,
                        "month": month,
                        "day": day

                    },
                },
            ],
        },
    }

    response = requests.post(URL, headers=HEADERS, json=body)
    response = response.json()['content']['results']

    if FILTER_DIRECT:
        response['quotes'] = {k: v for k, v in response['quotes'].items() if v['isDirect']}

    if FILTER_PRICE:
        response['quotes'] = {k: v for k, v in response['quotes'].items() if int(v['minPrice']['amount']) <= FILTER_PRICE}
    flights = []
    for flight_info in response['quotes'].values():
        assert flight_info['minPrice']['unit'] == 'PRICE_UNIT_WHOLE'
        flights.append(
            dict(
                price=int(flight_info['minPrice']['amount']),
                origin_place=get_place_from_id(flight_info['outboundLeg']['originPlaceId'], response['places']),
                destination_place=get_place_from_id(
                    flight_info['outboundLeg']['destinationPlaceId'], response['places']
                ),
                when=flight_info['outboundLeg']['departureDateTime'],
                destination_place_id=int(flight_info['outboundLeg']['destinationPlaceId']),
            )
        )
    return flights

def find_next_weekday_from_date(date: datetime.date, weekday: int) -> datetime.date:
    # Calculate the number of days to add to reach the next weekday (6 - current_day)
    days_until_saturday = (weekday - date.weekday() + 7) % 7
    
    # Calculate the next weekday date
    # if same day return same day
    return date + datetime.timedelta(days=days_until_saturday)

def get_from_destinations_flights(origin_grouped_flights: dict):
    all_flights = []
    with ThreadPoolExecutor() as executor:
        futures = []
        for place, flights_list in grouped_origin_flights.items():
            for flight in flights_list:
                for origin in ['WRO']: #, 'POZ', 'KTW', 'KRK']:
                    when = flight['when']
                    start_date = datetime.date(year=when['year'], month=when['month'], day=when['day'])
                    destination_id = flight['destination_place_id']
                    next_saturday = find_next_weekday_from_date(start_date, 5)
                    next_sunday = find_next_weekday_from_date(start_date, 6)
                    next_monday = find_next_weekday_from_date(start_date, 0)
                    for f_date in [next_saturday]: #, next_sunday, next_monday]:
                        f = executor.submit(
                            check_destination_to_origin_flights_from_date, 
                            f_date.year, f_date.month, f_date.day, 
                            destination_id, origin
                        )
                        futures.append(f)
        for future in as_completed(futures):
            all_flights.extend(future.result())
    return all_flights

destination_to_origin_flights = get_from_destinations_flights(grouped_origin_flights)